# Clustering Toronto Neighborhoods

## Part 1: Scraping Toronto Postal codes

To begin the creation of our table we need to import all the libraries (the ones that are going to be required in the upcoming analysis are included)

In [2]:
#Libraries
import pandas as pd
import numpy as np

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [41]:
! pip install folium==0.5.0
import folium

     |████████████████████████████████| 79 kB 9.8 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=4eb5d3a209599db218a32348178244cc2b5e21a08240ac08c565ade4ed8c62a0
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium


We then proceed to scrape the data we need from wikipedia, in order to obtain a table with all the features we need

In [3]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


We check the table for 'Not assigned' values for the Neighborhood and Borough features and we proceed to drop the lines without a specified Borough, as requested in the assignement.

In [4]:
pd.Series(df['Neighbourhood']=='Not assigned').value_counts()

False    103
True      77
Name: Neighbourhood, dtype: int64

In [5]:
pd.Series(df['Borough']=='Not assigned').value_counts()

False    103
True      77
Name: Borough, dtype: int64

In [6]:
df2 = df[df['Borough']!= 'Not assigned']
df2= df2.reset_index(drop=True)
df2

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


We finally verify that our table has no duplicate Postal Codes (so that neighborhoods are joined in the respective feature as required in the assignement) and we look for potential unassigned Neighborhoods.

In [7]:
duplicated=df2[['Postal Code']].duplicated()
pd.Series(duplicated).value_counts()

False    103
dtype: int64

In [8]:
pd.Series(df2['Neighbourhood']=='Not assigned').value_counts()

False    103
Name: Neighbourhood, dtype: int64

We check the shape of our table and notice that it's much bigger than what was required by the assignement, as it includes all Canadian postal codes, while only Toronto ones are required.

In [9]:
print('DF shape is:',df.shape)
print('DF2 shape is:',df2.shape)

DF shape is: (180, 3)
DF2 shape is: (103, 3)


We then proceed to filter our table for Toronto postal codes, manually reacreating the vector of the ones we were shown in the assignement.

In [25]:
Toronto_Codes = pd.DataFrame(['M5G','M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A'], columns=['Postal Code'])

In [26]:
Toronto_neigh = pd.DataFrame(pd.merge(Toronto_Codes,df2, on = 'Postal Code', how='inner'))
Toronto_neigh

,Postal Code,Borough,Neighbourhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


We proceed to check again the shape of our table and we verify that it matches with the requirements.

In [29]:
print('DF shape is:',Toronto_neigh.shape)
print('DF2 shape is:',Toronto_neigh.shape)

DF shape is: (12, 3)
DF2 shape is: (12, 3)


## Part 2: Attach coordinates

We proceed to scrape coordinates from the http link, as the geocoder returned an error, and we attach it to the previous dataframe.

In [30]:
coordinates=pd.read_csv('http://cocl.us/Geospatial_data')
coordinates

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [31]:
Toronto_neigh2=Toronto_neigh.merge(coordinates, on='Postal Code')
Toronto_neigh2

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


## Part 3: Explore and cluster Toronto's neighborhoods

In this last section we will have look at Toronto's neighborhoods and cluster them in accordance to what we have found out.

For starters, we will create a map of Toronto to get an idea of how the city looks like.

In [32]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [47]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(Toronto_neigh2['Latitude'], Toronto_neigh2['Longitude'], Toronto_neigh2['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

We can now proceed to collect further information about the neighborhoods by setting Foursquare credentials (hidden cell) and by creating a function that retrieves information about venues.

In [50]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: UQPJ1CLSUMES5JPOU3P5GAPCLYPURBK2KFQ0SE24GU3AYXVT
CLIENT_SECRET:12ULPSLKA4LJXVJZZP1ZI323OF0LUKMT4BQC544QAQY3TSCE


In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We now proceed to use the function to retrieve information and display the head of the resulting dataframe.

In [52]:
toronto_venues = getNearbyVenues(names=Toronto_neigh2['Neighbourhood'],
                                   latitudes=Toronto_neigh2['Latitude'],
                                   longitudes=Toronto_neigh2['Longitude']
                                  )

Central Bay Street
Hillcrest Village
Parkview Hill, Woodbine Gardens
Scarborough Village
Leaside
Studio District
Wexford, Maryvale
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
Humber Summit
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Malvern, Rouge
Regent Park, Harbourfront


In [75]:
print(toronto_venues.shape)
toronto_venues.head()

(230, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central Bay Street,43.657952,-79.387383,Jimmy's Coffee,43.658421,-79.385613,Coffee Shop
1,Central Bay Street,43.657952,-79.387383,Somethin' 2 Talk About,43.658395,-79.385338,Middle Eastern Restaurant
2,Central Bay Street,43.657952,-79.387383,Hailed Coffee,43.658833,-79.383684,Coffee Shop
3,Central Bay Street,43.657952,-79.387383,Tim Hortons,43.658570,-79.385123,Coffee Shop
4,Central Bay Street,43.657952,-79.387383,NEO COFFEE BAR,43.660130,-79.385830,Coffee Shop


We proceed to one hot encode the venue categories and attach them to the initial dataframe for the following clusterization.

In [81]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Sporting Goods Shop,Sports Bar,Stationery Store,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,Central Bay Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Central Bay Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Central Bay Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Central Bay Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Central Bay Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We group all the venues by neighbourhood, so that we will be able to cluster according to most common venues in the neighborhood.

In [84]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Sporting Goods Shop,Sports Bar,Stationery Store,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.0625,0.0625,0.125,0.125,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Central Bay Street,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.00000,0.016393,0.016393,0.000000,0.016393,0.016393,0.016393
2,Hillcrest Village,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Humber Summit,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Leaside,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000000,0.000000,0.000000,...,0.09375,0.03125,0.000000,0.03125,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Malvern, Rouge",0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Parkview Hill, Woodbine Gardens",0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Regent Park, Harbourfront",0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000000,0.022222,0.022222,...,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.044444,0.000000,0.000000,0.022222
8,Scarborough Village,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"South Steeles, Silverstone, Humbergate, Jamest...",0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [87]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [91]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport,Harbor / Marina,Plane,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden
1,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Burger Joint,Salad Place,Yoga Studio,Indian Restaurant,Modern European Restaurant
2,Hillcrest Village,Golf Course,Fast Food Restaurant,Pool,Mediterranean Restaurant,Athletics & Sports,Dog Run,Fish Market,Fish & Chips Shop,Coffee Shop,Comfort Food Restaurant
3,Humber Summit,Furniture / Home Store,Gym,Intersection,Yoga Studio,Dog Run,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop
4,Leaside,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Bank,Burger Joint,Shopping Mall,Fish & Chips Shop,Sandwich Place,Beer Store,Department Store


In [92]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 3, 0, 1, 0, 0, 4, 0], dtype=int32)

In [93]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_neigh2

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Burger Joint,Salad Place,Yoga Studio,Indian Restaurant,Modern European Restaurant
1,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,Golf Course,Fast Food Restaurant,Pool,Mediterranean Restaurant,Athletics & Sports,Dog Run,Fish Market,Fish & Chips Shop,Coffee Shop,Comfort Food Restaurant
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,0,Pizza Place,Café,Athletics & Sports,Flea Market,Gym / Fitness Center,Intersection,Pet Store,Pharmacy,Breakfast Spot,Bank
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Jewelry Store,Playground,Dog Run,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Department Store
4,M4G,East York,Leaside,43.709060,-79.363452,0,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Bank,Burger Joint,Shopping Mall,Fish & Chips Shop,Sandwich Place,Beer Store,Department Store


Let's display the final result on a map.

In [95]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters